In [5]:
import pandas as pd
from __future__ import print_function
import mlflow
import time

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.tracking.get_tracking_uri()
def now():
    now = int(time.time()+.5)
    dt = time.strftime("%Y-%m-%d_%H:%M:%S", time.gmtime(now))
    return dt

print("Run Start:",now())

Run Start: 2021-11-14_23:51:59


In [6]:
experiment_name = "Sales_Prediction"
Train = pd.read_csv('/Users/apple/Documents/Kaggle/ML_flow_projects/competitive-data-science-predict-future-sales/sales_train.csv')

In [7]:
import mlflow
print("MLflow Version:",mlflow.version.VERSION)
mlflow.set_experiment(experiment_name)
mlflow_client = mlflow.tracking.MlflowClient()
experiment_id = mlflow_client.get_experiment_by_name(experiment_name).experiment_id
print("experiment_id:",experiment_id)
print("experiment_name:",experiment_name)
run_origin = "jupyter"



MLflow Version: 1.21.0
experiment_id: 3
experiment_name: Sales_Prediction


In [8]:
import os
import warnings
import sys
import mlflow.sklearn
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import ElasticNet



def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [14]:
# Training Function

    
def train(alpha,ll_ratio):

    X = Train[['date_block_num','shop_id', 'item_id', 'item_cnt_day']]
    y = Train[['item_price']]



    # Split the data into training and test sets. (0.75, 0.25) split.
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.75, test_size=0.2, random_state=0)




    with mlflow.start_run() as run:
        run_id = run.info.run_uuid
        print("run_id:",run_id)
        print("run_origin:",run_origin)

        global clf
        clf = ElasticNet(alpha=alpha, l1_ratio=ll_ratio)


        clf.fit(X_train, y_train)

        predicted_qualities = clf.predict(X_valid)
        (rmse, mae, r2) = eval_metrics(y_valid, predicted_qualities)

        print("Elasticnet model (alpha={}, ll_ratio={}):".format(alpha, ll_ratio))
        print("  RMSE:",rmse)
        print("  MAE:",mae)
        print("  R2:",r2)

        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", ll_ratio)
        mlflow.log_param("run_origin", run_origin)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        mlflow.sklearn.log_model(clf, "model")

        #X = data.drop(["quality"], axis=1).values
        #y = data[["quality"]].values.ravel()
        #plot_file = "wine_quality.png"
        #plot_enet_descent_path(X, y, l1_ratio, plot_file)
        #mlflow.log_artifact(plot_file)
        

        return (rmse,r2,mae)

In [15]:
train(0.0001,1)

run_id: 9ca719753a8a4a35b68997200aa86bfe
run_origin: jupyter
Elasticnet model (alpha=0.0001, ll_ratio=1):
  RMSE: 1697.556286797524
  MAE: 724.8474941710072
  R2: 0.027875202370595153


(1697.556286797524, 0.027875202370595153, 724.8474941710072)

In [16]:
clf
import pickle
filename = 'sales_prediction_model.pkl'
pickle.dump(clf, open(filename, 'wb'))
